# Frictionless Pendulum on a Cart (Forcing)

## Imports & Symbols

In [1]:
import control
import matplotlib.pyplot as plt
import numpy as np
import scipy
import sympy as sp

np.set_printoptions(precision=4)
plt.rcParams["figure.figsize"] = [15, 8]

In [2]:
t, g = sp.symbols("t g")
l, m, M = sp.symbols("l m M")

theta = sp.Function("theta")
u = sp.Function("u")
x = sp.Function("x")

# System Modeling

In [3]:
x_p = x(t) + l * sp.sin(theta(t))
y_p = -l * sp.cos(theta(t))

x_p_dot = x_p.diff(t)
y_p_dot = y_p.diff(t)

V = -m * g * l * sp.cos(theta(t))
T = sp.Rational(1, 2) * M * (x(t).diff(t) ** 2) + sp.Rational(1, 2) * m * (
    x_p_dot**2 + y_p_dot**2
)

L = T - V

x_motion_eq, theta_motion_eq = sp.euler_equations(L, [x(t), theta(t)], t)

In [4]:
x_motion_eq = sp.Eq(x_motion_eq.lhs, u(t))
x_motion_eq

Eq(-M*Derivative(x(t), (t, 2)) - m*(-l*sin(theta(t))*Derivative(theta(t), t)**2 + l*cos(theta(t))*Derivative(theta(t), (t, 2)) + Derivative(x(t), (t, 2))), u(t))

In [5]:
theta_motion_eq.simplify()

Eq(l*m*(g*sin(theta(t)) + l*Derivative(theta(t), (t, 2)) + cos(theta(t))*Derivative(x(t), (t, 2))), 0)

In [6]:
sols = sp.solve(
    [x_motion_eq, theta_motion_eq], [x(t).diff(t, 2), theta(t).diff(t, 2)]
)

x_ddot = sols[x(t).diff(t, 2)].simplify()
x_ddot

(g*m*sin(2*theta(t))/2 + l*m*sin(theta(t))*Derivative(theta(t), t)**2 - u(t))/(M + m*sin(theta(t))**2)

In [7]:
theta_ddot = sols[theta(t).diff(t, 2)].simplify()
theta_ddot

(-M*g*sin(theta(t)) - g*m*sin(theta(t)) - l*m*sin(2*theta(t))*Derivative(theta(t), t)**2/2 + u(t)*cos(theta(t)))/(l*(M + m*sin(theta(t))**2))

In [8]:
x_ddot_sys = x_ddot.subs(u(t), 0)
x_ddot_sys

(g*m*sin(2*theta(t))/2 + l*m*sin(theta(t))*Derivative(theta(t), t)**2)/(M + m*sin(theta(t))**2)

In [9]:
theta_ddot_sys = theta_ddot.subs(u(t), 0)
theta_ddot_sys

(-M*g*sin(theta(t)) - g*m*sin(theta(t)) - l*m*sin(2*theta(t))*Derivative(theta(t), t)**2/2)/(l*(M + m*sin(theta(t))**2))

In [10]:
A_nl = sp.Matrix([x(t).diff(t), x_ddot_sys, theta(t).diff(t), theta_ddot_sys])
A_nl

Matrix([
[                                                                                                     Derivative(x(t), t)],
[                         (g*m*sin(2*theta(t))/2 + l*m*sin(theta(t))*Derivative(theta(t), t)**2)/(M + m*sin(theta(t))**2)],
[                                                                                                 Derivative(theta(t), t)],
[(-M*g*sin(theta(t)) - g*m*sin(theta(t)) - l*m*sin(2*theta(t))*Derivative(theta(t), t)**2/2)/(l*(M + m*sin(theta(t))**2))]])

In [11]:
x_ddot_u = -u(t) / (M + (m * sp.sin(theta(t)) ** 2))
x_ddot_u

-u(t)/(M + m*sin(theta(t))**2)

In [12]:
theta_ddot_u = (u(t) * sp.cos(theta(t))) / (
    l * (M + (m * sp.sin(theta(t)) ** 2))
)
theta_ddot_u

u(t)*cos(theta(t))/(l*(M + m*sin(theta(t))**2))

In [13]:
B_nl = sp.Matrix([0, x_ddot_u, 0, theta_ddot_u])
B_nl

Matrix([
[                                              0],
[                 -u(t)/(M + m*sin(theta(t))**2)],
[                                              0],
[u(t)*cos(theta(t))/(l*(M + m*sin(theta(t))**2))]])

# Linearization

In [14]:
sys_vars = [x(t), x(t).diff(t), theta(t), theta(t).diff(t)]
A_jacobian = A_nl.jacobian(sys_vars)
A_jacobian

Matrix([
[0, 1,                                                                                                                                                                                                                                                                                     0,                                                                    0],
[0, 0,                                                   -2*m*(g*m*sin(2*theta(t))/2 + l*m*sin(theta(t))*Derivative(theta(t), t)**2)*sin(theta(t))*cos(theta(t))/(M + m*sin(theta(t))**2)**2 + (g*m*cos(2*theta(t)) + l*m*cos(theta(t))*Derivative(theta(t), t)**2)/(M + m*sin(theta(t))**2), 2*l*m*sin(theta(t))*Derivative(theta(t), t)/(M + m*sin(theta(t))**2)],
[0, 0,                                                                                                                                                                                                                                                                               

In [15]:
B_jacobian = B_nl.jacobian([u(t)])
B_jacobian

Matrix([
[                                         0],
[               -1/(M + m*sin(theta(t))**2)],
[                                         0],
[cos(theta(t))/(l*(M + m*sin(theta(t))**2))]])

## Fixed Points

In [16]:
fixed_point = {x(t): 0, x(t).diff(t): 0, theta(t): sp.pi, theta(t).diff(t): 0}
Asym = A_jacobian.subs(fixed_point)
Asym

Matrix([
[0, 1,                 0, 0],
[0, 0,             g*m/M, 0],
[0, 0,                 0, 1],
[0, 0, (M*g + g*m)/(M*l), 0]])

In [17]:
fixed_point = {x(t): 0, x(t).diff(t): 0, theta(t): sp.pi, theta(t).diff(t): 0}
Bsym = B_jacobian.subs(fixed_point)
Bsym

Matrix([
[       0],
[    -1/M],
[       0],
[-1/(M*l)]])

# System Parameters

In [18]:
A = sp.lambdify([m, M, l, g], Asym)(0.005, 0.2, 300, 9807)
A

array([[  0.    ,   1.    ,   0.    ,   0.    ],
       [  0.    ,   0.    , 245.175 ,   0.    ],
       [  0.    ,   0.    ,   0.    ,   1.    ],
       [  0.    ,   0.    ,  33.5073,   0.    ]])

In [19]:
B = sp.lambdify([m, M, l, g], Bsym)(0.005, 0.2, 300, 9807)
B

array([[ 0.    ],
       [-5.    ],
       [ 0.    ],
       [-0.0167]])

# Controlability Analysis

# Controller Design (LQR)

In [20]:
Q = np.eye(4)
R = 0.0001

K, S, E = control.lqr(A, B, Q, R)
print(K, S, E, sep="\n\n")

[[ 1.0000e+02  1.3518e+02 -4.0771e+05 -7.1300e+04]]

[[ 1.3518e+00  4.1368e-01 -7.1300e+02 -1.2470e+02]
 [ 4.1368e-01  4.8651e-01 -8.3913e+02 -1.4677e+02]
 [-7.1300e+02 -8.3913e+02  1.4533e+06  2.5419e+05]
 [-1.2470e+02 -1.4677e+02  2.5419e+05  4.4457e+04]]

[-500.0034+0.j       -5.7175+0.0095j   -5.7175-0.0095j   -1.    +0.j    ]


In [21]:
np.savetxt("../pendulum/cart/lqr_params.csv", K)